# Exploratory Data Analysis & Model Fitting

In [ ]:
import os
import sys

# Add the src directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
FOLDS = 3

## Data Preprocessing

### Load the dataset

In [ ]:
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

from src import preprocessing

if "df_backup" not in globals():
    # takes 2-4min
    df = preprocessing.load_data()
    df_backup = df.copy()
else:
    df = df_backup.copy()  # type: ignore

### Brief Inspection

In [ ]:
df.info()

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df.dtypes)

In [ ]:
# View first few rows
print("First few rows:")
display(df.head())

# Check for missing values
print("\nMissing values:")
display(df.isnull().sum())

# Summary statistics
print("\nSummary statistics:")
display(df.describe(include='all'))

### Data Cleaning

In [ ]:
# Remove features that are independent of the target variable
INDEP_COLS = [
    'emp_title', 
    'id', 
    'member_id', 
    'policy_code', 
    'title', 
    'url',
]

# Remove columns that could cause data leakage
LEAKAGE_COLS = [
    'collection_recovery_fee', 
    'debt_settlement_flag', 
    'disbursement_method', 
    'funded_amnt_inv',
    'funded_amnt', 
    'hardship_flag',
    'initial_list_status',
    'issue_d',
    'last_credit_pull_d', 
    'last_fico_range_high',
    'last_fico_range_low', 
    'last_pymnt_amnt',
    'last_pymnt_d', 
    'next_pymnt_d',
    'out_prncp_inv', 
    'out_prncp',
    'pymnt_plan', 
    'recoveries', 
    'total_pymnt_inv', 
    'total_pymnt', 
    'total_rec_int',
    'total_rec_late_fee', 
    'total_rec_prncp', 
]

cols_to_drop = INDEP_COLS + LEAKAGE_COLS
df = preprocessing.drop_cols(df, cols=cols_to_drop)

In [ ]:
# Remove features with too many missing values
df = preprocessing.drop_sparse_cols(df, missing_rate=0.5)

In [ ]:
# Convert date columns to unix timestamps
date_cols = ['earliest_cr_line']
df = preprocessing.convert_dates(df, cols=date_cols)

In [ ]:
# Impute missing values
df = preprocessing.impute_missing_values(df)

In [ ]:
df.head()

### Categorical Encoding

In [ ]:
from src.feature_engineering import index_categories, frequency_encoding, onehot_encoding

# Encode target variable
df = index_categories(df, 'loan_status', categories=['Charged Off', 'Fully Paid'], drop=True)

In [ ]:
# Label Encoding for ordinal categorical variables
grades = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
df = index_categories(df, 'grade', categories=grades)

sub_grades = [f'{g}{s}' for g in grades for s in range(1, 6)]
df = index_categories(df, 'sub_grade', categories=sub_grades)

In [ ]:
# Encode low-cardinality categorical variables using one-hot encoding
ONEHOT_CATS = [
    'application_type',
    'home_ownership',
    'purpose',
    'term',
    'verification_status'
]

df = onehot_encoding(df, cols=ONEHOT_CATS)

In [ ]:
# Encode high-cardinality categorical variables using frequency encoding
FREQ_CATS = [
    'addr_state'
]

df = frequency_encoding(df, cols=FREQ_CATS)

### Drop highly correlated feature pairs

In [ ]:
from src import algebra, visualize
from src.feature_engineering import drop_corr_pairs

cols = df.columns.values
corr = algebra.correlation_matrix(df)

# plot correlation matrix
visualize.correlation_matrix(corr)

df = drop_corr_pairs(df, corr)

removed = np.setdiff1d(cols, df.columns.values)
print(f"Removed columns:\n\t{"\n\t".join(removed)}")

### Feature Engineering

In [ ]:
from src.feature_engineering import extract_integers, frequency_encoding, new_features

df = extract_integers(df, cols=['emp_length', 'zip_code'])

### Scale feature values

In [ ]:
from src.preprocessing import scale_features

df = scale_features(df)

### Take another peak

In [ ]:
df.head()

### Handle data imbalance

In [ ]:
visualize.class_dist(df)

In [ ]:
from src import training

X_resampled, y_resampled = training.resample(df)

### Split the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

### Feature Selection using RFE

In [ ]:
from src.model import create_model

rfe_model = create_model(eval_metric='logloss')

if "n_features_to_select" in globals():
    from sklearn.feature_selection import RFE

    # Create a model to determine the most important features
    rfe = RFE(
        estimator=rfe_model,
        n_features_to_select=n_features_to_select # type: ignore
    )
    rfe.fit(X_train, y_train)
else:
    from sklearn.feature_selection import RFECV

    # Create a model to determine the number of features for feature selection
    rfe = RFECV(
        estimator=rfe_model, cv=FOLDS, scoring='roc_auc', n_jobs=(-1))
    rfe.fit(X_train, y_train)
    n_features_to_select = rfe.n_features_

    # Plot rfe.cv_results_
    plt.figure()
    plt.title("RFE with cross-validation")
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score")
    plt.plot(rfe.cv_results_["n_features"], rfe.cv_results_["mean_test_score"])
    cv_score = rfe.cv_results_["mean_test_score"][n_features_to_select - 1]
    plt.scatter(n_features_to_select, cv_score, marker="|", s=100)
    plt.show()

score = rfe.score(X_test, y_test)
print(f"Optimal number of features to select: {n_features_to_select}") # type: ignore
print(f"RFE Test Score: {score}")

In [ ]:
cols = X_train.columns[rfe.support_]
X_train = pd.DataFrame(rfe.transform(X_train), columns=cols) # type: ignore
X_test = pd.DataFrame(rfe.transform(X_test), columns=cols) # type: ignore

In [ ]:
X_test.head()

## Training

### Train the model

In [ ]:
model = create_model(eval_metric='logloss')

model.fit(X_train, y_train)

## Inference
### Make predictions

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

## Evaluation

### Accuracy

In [ ]:
# Accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

### Classification Report

In [ ]:
# Classification report
from sklearn.metrics import classification_report

print("Classification Report:\n", classification_report(y_test, y_pred))

### Confusion Matrix

In [ ]:
# Confusion matrix
visualize.confusion_matrix(y_test, y_pred)

### ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score

# ROC-AUC score
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC Score: {roc_auc:.4f}")

# Plot ROC curve
visualize.roc_curve(y_test, y_proba, roc_auc)

### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')

print(f"Cross-Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean Cross-Validation ROC-AUC Score: {cv_scores.mean():.4f}")

## Feature importance

In [ ]:
# calculate correlations between the features in X_train and the target variable y_train
df_train = pd.concat([X_train, y_train])
correlations = df_train.corr()['loan_status'].sort_values().to_frame()

# plot the correlations
visualize.correlations(correlations.drop('loan_status'))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.inspection import permutation_importance

# Calculate permutation importance
result = permutation_importance(
    model, X_train, y_train, n_repeats=FOLDS, random_state=42)

# Get importance values
importances = result['importances_mean']

# Create a DataFrame with feature names and importances
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
})

# Sort the DataFrame by importance in descending order
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Display the feature importances DataFrame
with pd.option_context('display.max_rows', None):
    display(feature_importances)


## Hyperparameter Tuning

### HP search

In [ ]:
import random

from sklearn.model_selection import RandomizedSearchCV
from scipy import stats


param_dist = {
    "max_depth": stats.randint(2, 3),
    "learning_rate": stats.uniform(loc=0.93, scale=0.07),
    "n_estimators": stats.randint(100, 1000),
    "subsample": stats.norm(0.85, scale=0.05),
    "colsample_bytree": stats.uniform(loc=0.98, scale=0.02),
}

# Setup the randomized search with 50 iterations
search = RandomizedSearchCV(
    estimator=create_model(eval_metric='logloss'),
    param_distributions=param_dist,
    n_iter=30, # ~9sec/iter
    cv=FOLDS,
    verbose=0,
    random_state=42,
    n_jobs=(-1)
)

# Fit the model
search.fit(X_train, y_train)

# Best Model
print("Best Parameters:", search.best_params_)
print(f"Best ROC-AUC Score: {search.best_score_:.8f}")

### Retrain with these hyperparameters

In [ ]:
best_params = search.best_params_
model_best = create_model(**best_params)
model_best.fit(X_train, y_train)

### Evaluate the optimized model

In [ ]:
# Predict on test data
y_pred_best = model_best.predict(X_test)
y_proba_best = model_best.predict_proba(X_test)[:,1]

# Accuracy
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Optimized Accuracy: {accuracy_best:.4f}")

print("Optimized Classification Report:\n", classification_report(y_test, y_pred_best))

In [ ]:
# ROC-AUC score
roc_auc = roc_auc_score(y_test, y_proba_best)
print(f"Optimized ROC-AUC Score: {roc_auc:.4f}")

# Plot ROC curve
visualize.roc_curve(y_test, y_proba_best, roc_auc)